# Input: train and val preprocessed data

In [22]:
# To Do, replace this
train_ds = 0
val_ds = 0


# Imports

In [16]:
import os
from face_tally.params import *

import tensorflow as tf
from tensorflow import keras

import keras_cv
from keras_cv import bounding_box, visualization
from ultralytics import YOLO


# Define the number of classes for the model

In [5]:
class_ids = ['face']
class_mapping = dict(zip(range(len(class_ids)), class_ids))


# Import a backone for the model. Start with small one from coco

In [12]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_s_backbone_coco"  # We will use yolov8 small backbone with coco weights
)


20596968/20596968 [==============================] - 2s 0us/step


# Define YOLO model with face class and COCO backbone

In [13]:
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="xyxy",
    backbone=backbone,
    fpn_depth=1,
)


# Compile the model with binary_crossentropy and Complete IoU (CIoU) metric

In [17]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    global_clipnorm=GLOBAL_CLIPNORM,
)

yolo.compile(
    optimizer=optimizer, classification_loss="binary_crossentropy", box_loss="ciou"
)


In [ ]:
yolo.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)


In [ ]:
def train_model():
    # Load the pre-trained model
    model = YOLO("yolov8s-p2.yaml").load("yolov8s.pt")

    # Train the model
    model.train(
        data="dataset.yaml", epochs=200,  imgsz=256, save=True, format='onnx'
    )  # Set imgsz to 256 for training on 256x256 images

    # Export the model to ONNX format
    path = model.export()
    print(path)


In [ ]:
# Example of predictions
model = YOLO('best-30.pt') # Load this from
results = model('yolo/images/test/image_4008.png', imgsz=256, save=True, conf=0.5)


In [18]:
def visualize_detections(model, dataset, bounding_box_format):
    images, y_true = next(iter(dataset.take(1)))
    y_pred = model.predict(images)
    y_pred = bounding_box.to_ragged(y_pred)
    visualization.plot_bounding_box_gallery(
        images,
        value_range=(0, 255),
        bounding_box_format=bounding_box_format,
        y_true=y_true,
        y_pred=y_pred,
        scale=4,
        rows=2,
        cols=2,
        show=True,
        font_scale=0.7,
        class_mapping=class_mapping,
    )


In [ ]:
visualize_detections(yolo, dataset=val_ds, bounding_box_format="xyxy")
